In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
train = pd.read_csv('/content/Train.csv')
test = pd.read_csv('/content/Test.csv')
sub = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
y = train.score.values

In [ ]:
train.head(30)

,ID,content,score
0,113733,حلو جدا اوى خالص,1
1,10380,Je n arrive pas à me connecter sur l applicati...,0
2,120608,Alimentation de la carte n est pas tjs disponible,0
3,126590,gcgcc,1
4,86818,معقول بس البحث فيه سئ,1
5,80707,هذا من فضل الله,1
6,131938,Bonnz app,1
7,7038,Ja jure aujourd hui il y a mega hamza vendredi...,-1
8,136644,"Bonjour , Je narrive pas a ajouté des bénéfic...",-1
9,11706,Bon application continue,1


In [ ]:
train['len_text'] = train['content'].apply(len)
test['len_text'] = test['content'].apply(len)

In [ ]:
test.describe()

,ID,len_text
count,19803.000000,19803.000000
mean,64822.685098,55.590971
std,44336.687355,71.707666
min,2.000000,1.000000
25%,21825.500000,15.000000
50%,64969.000000,31.000000
75%,105200.000000,66.000000
max,142720.000000,1098.000000


In [ ]:
fr_stop = 'alors au aucuns aussi autre avant avec avoir bon car ce cela ces ceux chaque ci comme comment dans des du dedans dehors depuis devrait doit donc dos début elle elles en encore essai est et eu fait faites fois font hors ici il ils je	juste la le les leur là ma maintenant mais mes mien moins mon mot même ni nommés notre nous ou où par parce pas peut peu plupart pour pourquoi quand que quel quelle quelles quels qui sa sans ses seulement si sien son sont sous soyez	sujet sur ta tandis tellement tels tes ton tous tout trop très tu voient vont votre vous vu ça étaient état étions été être'.split()
ar_stop = '، ء ءَ آ آب آذار آض آل آمينَ آناء آنفا آه آهاً آهٍ آهِ أ أبدا أبريل أبو أبٌ أجل أجمع أحد أخبر أخذ أخو أخٌ أربع أربعاء أربعة أربعمئة أربعمائة أرى أسكن أصبح أصلا أضحى أطعم أعطى أعلم أغسطس أفريل أفعل به أفٍّ أقبل أكتوبر أل ألا ألف ألفى أم أما أمام أمامك أمامكَ أمد أمس أمسى أمّا أن أنا أنبأ أنت أنتم أنتما أنتن أنتِ أنشأ أنه أنًّ أنّى أهلا أو أوت أوشك أول أولئك أولاء أولالك أوّهْ أى أي أيا أيار أيضا أيلول أين أيّ أيّان أُفٍّ ؤ إحدى إذ إذا إذاً إذما إذن إزاء إلى إلي إليكم إليكما إليكنّ إليكَ إلَيْكَ إلّا إمّا إن إنَّ إى إياك إياكم إياكما إياكن إيانا إياه إياها إياهم إياهما إياهن إياي إيهٍ ئ ا ا? ا?ى االا االتى ابتدأ ابين اتخذ اثر اثنا اثنان اثني اثنين اجل احد اخرى اخلولق اذا اربعة اربعون اربعين ارتدّ استحال اصبح اضحى اطار اعادة اعلنت اف اكثر اكد الآن الألاء الألى الا الاخيرة الان الاول الاولى التى التي الثاني الثانية الحالي الذاتي الذى الذي الذين السابق الف اللاتي اللتان اللتيا اللتين اللذان اللذين اللواتي الماضي المقبل الوقت الى الي اليه اليها اليوم اما امام امس امسى ان انبرى انقلب انه انها او اول اي ايار ايام ايضا ب بؤسا بإن بئس باء بات باسم بان بخٍ بد بدلا برس بسبب بسّ بشكل بضع بطآن بعد بعدا بعض بغتة بل بلى بن به بها بهذا بيد بين بَسْ بَلْهَ ة ت تاء تارة تاسع تانِ تانِك تبدّل تجاه تحت تحوّل تخذ ترك تسع تسعة تسعمئة تسعمائة تسعون تسعين تشرين تعسا تعلَّم تفعلان تفعلون تفعلين تكون تلقاء تلك تم تموز تينك تَيْنِ تِه تِي ث ثاء ثالث ثامن ثان ثاني ثلاث ثلاثاء ثلاثة ثلاثمئة ثلاثمائة ثلاثون ثلاثين ثم ثمان ثمانمئة ثمانون ثماني ثمانية ثمانين ثمنمئة ثمَّ ثمّ ثمّة ج جانفي جدا جعل جلل جمعة جميع جنيه جوان جويلية جير جيم ح حاء حادي حار حاشا حاليا حاي حبذا حبيب حتى حجا حدَث حرى حزيران حسب حقا حمدا حمو حمٌ حوالى حول حيث حيثما حين حيَّ حَذارِ خ خاء خاصة خال خامس خبَّر خلا خلافا خلال خلف خمس خمسة خمسمئة خمسمائة خمسون خمسين خميس د دال درهم درى دواليك دولار دون دونك ديسمبر دينار ذ ذا ذات ذاك ذال ذانك ذانِ ذلك ذهب ذو ذيت ذينك ذَيْنِ ذِه ذِي ر رأى راء رابع راح رجع رزق رويدك ريال ريث رُبَّ ز زاي زعم زود زيارة س ساء سابع سادس سبت سبتمبر سبحان سبع سبعة سبعمئة سبعمائة سبعون سبعين ست ستة ستكون ستمئة ستمائة ستون ستين سحقا سرا سرعان سقى سمعا سنة سنتيم سنوات سوف سوى سين ش شباط شبه شتانَ شخصا شرع شمال شيكل شين شَتَّانَ ص صاد صار صباح صبر صبرا صدقا صراحة صفر صهٍ صهْ ض ضاد ضحوة ضد ضمن ط طاء طاق طالما طرا طفق طَق ظ ظاء ظل ظلّ ظنَّ ع عاد عاشر عام عاما عامة عجبا عدا عدة عدد عدم عدَّ عسى عشر عشرة عشرون عشرين عل علق علم على علي عليك عليه عليها علًّ عن عند عندما عنه عنها عوض عيانا عين عَدَسْ غ غادر غالبا غدا غداة غير غين ـ ف فإن فاء فان فانه فبراير فرادى فضلا فقد فقط فكان فلان فلس فهو فو فوق فى في فيفري فيه فيها ق قاطبة قاف قال قام قبل قد قرش قطّ قلما قوة ك كأن كأنّ كأيّ كأيّن كاد كاف كان كانت كانون كثيرا كذا كذلك كرب كسا كل كلتا كلم كلَّا كلّما كم كما كن كى كيت كيف كيفما كِخ ل لأن لا لا سيما لات لازال لاسيما لام لايزال لبيك لدن لدى لدي لذلك لعل لعلَّ لعمر لقاء لكن لكنه لكنَّ للامم لم لما لمّا لن له لها لهذا لهم لو لوكالة لولا لوما ليت ليرة ليس ليسب م مئة مئتان ما ما أفعله ما انفك ما برح مائة ماانفك مابرح مادام ماذا مارس مازال مافتئ ماي مايزال مايو متى مثل مذ مرّة مساء مع معاذ معه مقابل مكانكم مكانكما مكانكنّ مكانَك مليار مليم مليون مما من منذ منه منها مه مهما ميم ن نا نبَّا نحن نحو نعم نفس نفسه نهاية نوفمبر نون نيسان نيف نَخْ نَّ ه هؤلاء ها هاء هاكَ هبّ هذا هذه هل هللة هلم هلّا هم هما همزة هن هنا هناك هنالك هو هي هيا هيهات هيّا هَؤلاء هَاتانِ هَاتَيْنِ هَاتِه هَاتِي هَجْ هَذا هَذانِ هَذَيْنِ هَذِه هَذِي هَيْهات و و6 وأبو وأن وا واحد واضاف واضافت واكد والتي والذي وان واهاً واو واوضح وبين وثي وجد وراءَك ورد وعلى وفي وقال وقالت وقد وقف وكان وكانت ولا ولايزال ولكن ولم وله وليس ومع ومن وهب وهذا وهو وهي وَيْ وُشْكَانَ ى ي ياء يفعلان يفعلون يكون يلي يمكن يمين ين يناير يوان يورو يوليو يوم يونيو ّأيّان'.split()

In [ ]:
#cleaning text

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import re
def clean_text(text):
  ps = PorterStemmer() 
  toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
  message = []
  #convertir tous minuscule
  review = text.lower()
  #splitter chaque ligne
  review = toker.tokenize(review)
  #
  review = [ps.stem(word) for word in review if not word in fr_stop]
  review = [word for word in review if not word in ar_stop]
  review = [word for word in review if not word.isdigit()]
  #construire de nouveau la ligne
  res = ' '.join(review)
  return res

train['clean_content'] = train['content'].apply(clean_text)
test['clean_content'] = test['content'].apply(clean_text)

In [ ]:
import nltk
nltk.download('punkt')
train_df = train
test_df = test

count_vec = CountVectorizer(tokenizer=word_tokenize,token_pattern=None)
count_vec.fit(train_df.clean_content)

xtrain = count_vec.transform(train_df.clean_content)
xtest = count_vec.transform(test_df.clean_content)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model = linear_model.LogisticRegression(max_iter=1000)
model.fit(xtrain, train_df.score)
preds = model.predict(xtest)

In [ ]:
sub.score = preds
sub.to_csv('submission5.csv', index=False)

In [ ]:
xtrain.shape

(59408, 55561)

In [ ]:
xtest.shape

(19803, 55561)

In [ ]:
#modéle2

In [ ]:
y

array([1, 0, 0, ..., 1, 1, 1])

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(xtrain,y, test_size = 0.2, random_state = 42)
print(X_train1.shape,Y_train1.shape)
print(X_test1.shape,Y_test1.shape)

(47526, 55561) (47526,)
(11882, 55561) (11882,)


In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC(C=100,kernel='rbf',gamma=0.001)

In [ ]:
model.fit(X_train1,Y_train1)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = model.predict(X_test1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,Y_test1)

0.7923750210402289

In [ ]:
y_pred = model.predict(xtest)
sub.score = y_pred
sub.to_csv('submission6.csv', index=False)

In [ ]:
##########################################""

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train['clean_content'].values)
X1 = tokenizer.texts_to_sequences(train['clean_content'].values)
X2 = tokenizer.texts_to_sequences(test['clean_content'].values)


In [ ]:
df1 = train[['ID','clean_content']].copy()
df2 = test[['ID','clean_content']].copy()

In [ ]:
df1.shape

(59408, 2)

In [ ]:
df2.shape

(19803, 2)

In [ ]:
df3 = pd.concat([df1,df2])
df3.shape

(79211, 2)

In [ ]:
df3.head()

,ID,clean_content
0,113733,حلو اوى خالص
1,10380,n arriv à me connect ...
2,120608,aliment de cart n ...
3,126590,gcgcc
4,86818,معقول بس البحث سئ


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 3000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df3['clean_content'].values)
X = tokenizer.texts_to_sequences(df3['clean_content'].values)
X = pad_sequences(X)



In [ ]:
X.shape

(79211, 150)

In [ ]:
X1 = X[:59408]

In [ ]:
X2 = X[59408:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 150, 128)          384000    
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 150, 128)          0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 196)               254800    
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 591       
Total params: 639,391
Trainable params: 639,391
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y1 = pd.get_dummies(train['score']).values
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1, test_size = 0.25, random_state = 42)
print(X_train1.shape,Y_train1.shape)
print(X_test1.shape,Y_test1.shape)


(44556, 150) (44556, 3)
(14852, 150) (14852, 3)


In [ ]:
batch_size = 64
model.fit(X_train1, Y_train1, epochs = 15, batch_size=batch_size,validation_data=(X_test1,Y_test1))

Epoch 1/15
697/697 [==============================] - 710s 1s/step - loss: 0.6110 - accuracy: 0.7698 - val_loss: 0.5542 - val_accuracy: 0.7933
Epoch 2/15
697/697 [==============================] - 714s 1s/step - loss: 0.5264 - accuracy: 0.8069 - val_loss: 0.5518 - val_accuracy: 0.7923
Epoch 3/15
697/697 [==============================] - 714s 1s/step - loss: 0.5045 - accuracy: 0.8146 - val_loss: 0.5464 - val_accuracy: 0.7934
Epoch 4/15
697/697 [==============================] - 772s 1s/step - loss: 0.4880 - accuracy: 0.8191 - val_loss: 0.5567 - val_accuracy: 0.7917
Epoch 5/15
697/697 [==============================] - 784s 1s/step - loss: 0.4743 - accuracy: 0.8239 - val_loss: 0.5701 - val_accuracy: 0.7908
Epoch 6/15
697/697 [==============================] - 781s 1s/step - loss: 0.4619 - accuracy: 0.8275 - val_loss: 0.5873 - val_accuracy: 0.7892
Epoch 7/15
697/697 [==============================] - 777s 1s/step - loss: 0.4504 - accuracy: 0.8320 - val_loss: 0.6011 - val_accuracy: 0.7895

In [ ]:
res = model.predict(X2)

In [ ]:
res

array([[9.51933682e-01, 3.88375372e-02, 9.22881067e-03],
       [9.99961853e-01, 2.96823418e-05, 8.48424679e-06],
       [9.99809682e-01, 3.20253312e-05, 1.58335257e-04],
       ...,
       [4.12136972e-01, 1.45696034e-03, 5.86406052e-01],
       [1.15172796e-01, 1.02080489e-02, 8.74619186e-01],
       [9.35984254e-01, 1.46246911e-03, 6.25533238e-02]], dtype=float32)

In [ ]:
import numpy as np

In [ ]:
resf = np.argmax(res,axis=1)

In [ ]:
test_id = test.ID
sub1 = pd.DataFrame()
sub1['ID'] = test_id
sub1['score'] = resf
sub1['score'] = sub1['score'].replace({ 0 : -1 ,1 : 0 , 2 : 1})

In [ ]:
sub1.to_csv('subb.csv', index=False)

In [ ]:
result = model.predict(X2,batch_size=1,verbose = 2)

19803/19803 - 611s


In [ ]:
sub.head()

,ID,score
0,2,0.765397
1,8,0.964965
2,10,0.970238
3,12,0.461654
4,13,0.582706


In [ ]:
sub.describe()

,ID,score
count,19803.000000,19803.000000
mean,64822.685098,0.291689
std,44336.687355,0.334108
min,2.000000,0.000492
25%,21825.500000,0.026110
50%,64969.000000,0.139254
75%,105200.000000,0.444375
max,142720.000000,0.999867


In [ ]:
sub1 = sub.copy()

In [ ]:
def eval(x):
  if x <= 0.333333:
    return 1
  elif x <= 0.666666:
    return 0
  else:
    return -1

In [ ]:
sub1['score'] = sub1['score'].apply(eval)

In [ ]:
sub1.head()

,ID,score
0,2,-1
1,8,-1
2,10,-1
3,12,0
4,13,0


In [ ]:
sub1.to_csv('sub12.csv', index=False)